In [ ]:
#| default_exp connect

# Connect -- Access the pklmart database

> connect to the pklmart database

In practice, you can probably just ignore this module. pklshop includes the data from pklmart already loaded into pandas dataframes. Connect is only needed if you want to access the database directly and have permission. It is also used to update the dataframes in pklshop periodically.

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from configparser import ConfigParser
import psycopg2
import pandas as pd

In [ ]:
#| export
def config(filename="../database.ini", section="postgresql"):
    """Reads the database.ini file and returns the connection parameters as a dictionary.
    Assumes the ini file is in the parent directory"""
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        # raise Exception(
        #     "Section {0} not found in the {1} file".format(section, filename)
        # )
        print("Section {0} not found in the {1} file".format(section, filename))
        
    return db

In [ ]:
show_doc(config)

---

[source](https://github.com/NolanSmyth/pklshop/blob/main/pklshop/connect.py#L9){target="_blank" style="float:right; font-size:smaller"}

### config

>      config (filename='../database.ini', section='postgresql')

Reads the database.ini file and returns the connection parameters as a dictionary.
Assumes the ini file is in the parent directory

You will now want to set up a `database.ini` file. This will contain the sensitive information needed to connect to the databse. To avoid this information being public, be sure to add `*.ini` to your `.gitignore`. Once completed, it should look something like this:

#### **`database.ini`**
```
[postgresql]
database=DATABASE
host=HOST
user=USERNAME
password=PASSWORD
port=PORT
```
where you replace the uppercase values with the appropriate information.

We can now use the config function to get the appropriate information:

In [ ]:
params = config()

We can now connect to the database using a `DbConnection` object

In [ ]:
#| export
class DbConnection:
    "Class to create a connection to the database"
    def __init__(self, 
                params:dict):
        #todo add checks for params type and values
        if not isinstance(params, dict):
            raise TypeError(f"params must be a dictionary")
        self.params = params 
        self.conn = None
    def __str__(self): return f"connection object"
    __repr__ = __str__

    def _connect(self):
        try:
            self.conn = psycopg2.connect(**self.params)
        except:
            print("Unable to connect to the database.")

    def _close(self):
        if self.conn is not None:
            self.conn.close()
        else:
            print("Connection already closed. No action taken.")

    def pull_data(self, tablename):

        """
        Pulls data from the database for a given table and returns a pandas dataframe.
        See https://pklmart.com/postgres_pklm_pklm_prd/relationships.html for a list of tables.
        """
        #Open the connection
        self._connect()

        with self.conn.cursor() as cursor:
            try:
                cursor.execute(
                    f"SELECT column_name FROM information_schema.columns where table_name='{tablename}';"
                )
                cols = cursor.fetchall()
                cols = [cols[x][0] for x in range(len(cols))]
                cursor.execute(f"SELECT * FROM pklm_prd.{tablename}")
                data = cursor.fetchall()
                self._close()
                return pd.DataFrame(data, columns=cols)
            except Exception as inst:
                print(type(inst))
                print(inst.args)
                print(inst)

We can create a `DbConnection` like this:

In [ ]:
conn = DbConnection(params)

Now we want to be able to pull data from the database to use and analyze. We can do this by passing our connection and table name to the pull_data function. Look at the data notebook to see this in action

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()